In [1]:
import numpy as np
import pandas as pd

import pickle
import sys

from scipy.sparse.csr import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import model_from_json

Using TensorFlow backend.


In [143]:
def saveKerasModel(modelName, model):
    model_json = model.to_json()
    with open(modelName, "w") as json_file:
        json_file.write(model_json)
def loadKerasModel(modelName):
    json_file = open(modelName, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model

In [9]:
!sudo aws s3 cp s3://RecipeVectors/unique_ingredients.pkl .
!sudo aws s3 cp s3://RecipeVectors/CleanedIngredients.pkl .

download: s3://RecipeVectors/unique_ingredients.pkl to ./unique_ingredients.pkl


In [10]:
with open('unique_ingredients.pkl', 'rb') as f:
    unique_ingredients = pickle.load(f)

In [13]:
df = pd.read_pickle('CleanedIngredients.pkl')

# Classification Model 1, Categories Clustering as Label

Since a recipe can belong to multiple categories, and it is logical for a recipe to belong in multiple categories(i.e. a medium-rare steak can fit in following categories: beef, dinner, meat etc.) We first cluster the categories and assign each recipe with its respective index. 

In the subsequent classification, we will use the cluster index as the Y label, and then retrieve the centroid corresponding to the cluster index to output the general categories

In [324]:
from nltk.stem.snowball import *

stemmer = SnowballStemmer('english')
USELESS_CATEGORIES = set(map(stemmer.stem, ['recipes', 'and','main','dishes', 'cooking']))

def removeUselessCategories(arr):
    
    n_arr = []
    for category in arr:
        st_cat = stemmer.stem(category)
        for word in st_cat.split():
            if not word in USELESS_CATEGORIES:
                n_arr.append(word)
    return ' , '.join(n_arr)
df_cluster = df.copy()
df_cluster['allCategories'] = df_cluster['categories'].apply(removeUselessCategories)
df_cluster['allCategories'].sample(5)

13592    beef , appet , meat , poultry , appet , appeti...
6638     japanese , asian , salmon , fish , everyday , ...
3452     scandinavian , norwegian , norwegian , cooki ,...
451      squash , side , roasted , veget , vegetable , ...
3828     roasted , veget , brussels , sprouts , side , ...
Name: allCategories, dtype: object

In [44]:
def makePipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(stop_words='english')),
        ('tfidf', TfidfTransformer()),
        ('clf', KMeans(n_clusters=20, init='k-means++', max_iter=100, n_init=1))
    ])
    return pipeline

def trainModel(X):
    model = makePipeline()
    model.fit(X)
    return model

model = trainModel(df_cluster['allCategories'].values)
print("Top terms per cluster:")

vectorizer = model.named_steps['vect']
km = model.named_steps['clf']
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

true_k = 20
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print "Cluster %d:" % i
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind]
    print

Top terms per cluster:
Cluster 0:
 cooki
 dessert
 fruit
 international
 drop
 bar
 christmas
 italian
 chocolate
 nut

Cluster 1:
 appet
 spread
 dips
 cheese
 snack
 appetizers
 salsa
 dip
 mexican
 bean

Cluster 2:
 pasta
 shap
 italian
 minute
 chees
 lasagna
 everyday
 macaroni
 vegetarian
 30

Cluster 3:
 soup
 soups
 chili
 stew
 stews
 canned
 chicken
 pea
 everyday
 beef

Cluster 4:
 chicken
 breast
 baked
 roasted
 everyday
 low
 asian
 style
 minute
 pasta

Cluster 5:
 seafood
 everyday
 vegetable
 low
 potato
 fish
 canned
 salmon
 squash
 mexican

Cluster 6:
 vegetarian
 indian
 vegan
 pea
 bean
 grain
 everyday
 calorie
 asian
 tofu

Cluster 7:
 cooki
 dessert
 chocolate
 christmas
 extract
 vanilla
 browni
 drop
 bar
 extracts

Cluster 8:
 breakfast
 brunch
 egg
 potato
 seafood
 pancak
 meat
 casserol
 bread
 everyday

Cluster 9:
 cak
 dessert
 cake
 mix
 chocolate
 holiday
 fruit
 pumpkin
 cupcak
 everyday

Cluster 10:
 dress
 stuffing
 thanksgiving
 sausage
 fall
 bre

In [47]:
# apply the cluster index to the model
df['category_cluster_index'] = model.predict(df['allCategories'].values)

In [49]:
df['category_cluster_index'].sample(5)

3960      2
14340     9
2202      2
3933      2
2256     11
Name: category_cluster_index, dtype: int32

## train model on the 20 cluster indexes
Here we are using the cluster indices as labels and concating the ingredient list together to create the features. We then train a RNN classification model. The first layer of our sequential model use the embedding of size 100, the second layer is a LSTM layer, we also added a dropout layer of 0.4. The final layer is a softmax layer to map it to one of our 20 indices. 

We run the model for 10 epochs and achieved an accuracy of 0.78. 

In [51]:
def concatIngredients(arr):
    return ','.join(arr).encode('ascii', 'ignore')

def processKerasModel(df_):
    embedding_length = 100
    top_words = 10000
    df = df_.copy()
    df['features'] = df['ingredients'].apply(concatIngredients)
    df['label'] = df['category_cluster_index']
    df = df[df['label'].astype(str) != 'nan']
    le = preprocessing.LabelEncoder()
    features = df['features'].values
    labels = df['label'].values
    
    le.fit(np.unique(labels))
    print list(le.classes_), 'num of labels:',len(np.unique(labels))
    labels = le.transform(labels) 
    print labels[:10]
#     X_train, Y_train, X_test, Y_test = ut.simpleSplit(features, labels)

    tokenizer = Tokenizer(nb_words=top_words)
    tokenizer.fit_on_texts(features)
    sequences = tokenizer.texts_to_sequences(features)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=embedding_length)

#     labels = np_utils.to_categorical(np.asarray(labels))
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    # split the data into a training set and a validation set
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices]
    labels = labels[indices]
    nb_validation_samples = int(0.2 * data.shape[0])

    X_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    X_test = data[-nb_validation_samples:]
    y_test = labels[-nb_validation_samples:]
    
    embedding_vecor_length = 100
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=embedding_length))
#     model.add(Dropout(0.5))
    model.add(LSTM(100)) #, return_sequences=True))
    model.add(Dropout(0.4))
#     model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(len(np.unique(labels)), activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])
    print(model.summary())
    
    model.fit(X_train, y_train, epochs=10, batch_size=100)
    # Final evaluation of the model
    return model, X_test, y_test

rnn_cluster_model, X_test, y_test = processKerasModel(df_cluster)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] num of labels: 20
[ 1  1  5  0  4  3 16  6  4 16]


/Users/chuqinghe/anaconda/envs/py27/lib/python2.7/site-packages/keras/preprocessing/text.py:139: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 5486 unique tokens.
('Shape of data tensor:', (89061, 100))
('Shape of label tensor:', (89061,))
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2020      
Total params: 1,082,420
Trainable params: 1,082,420
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
71249/71249 [==============================] - 474s - loss: 2.1189 - acc: 0.3357   
Epoch 2/10
71249/71249 [==============================] - 419

The model accuracy on the train set is 0.78, the accuracy on the test set is 0.75. In total the model took about 2 hours to train

In [145]:
# save the model for future use
saveKerasModel('model_top_20_clustering_index.json',rnn_cluster_model)

In [325]:
# Load model if needed
# t_model = loadKerasModel('model_top_20_clustering_index.json')
# t_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Classification  Model 2: Top 30 Categories by Index
In this model, we have first extract the top 30 categories by doing counting the occurency of each category. Then we set the label of each recipe to one of the top 30 categories.  

In [330]:
from collections import Counter
unique_cat = Counter()
USELESS_CATEGORIES = ['recipes']
for idx, row in df.iterrows():
    for cat in row['categories']:
        cat = cat.lower()
        if not cat in USELESS_CATEGORIES:
            unique_cat[cat]+=1

In [331]:
TOP_K_CAT = 30
k_cat = [cat[0] for cat in unique_cat.most_common(TOP_K_CAT)]
k_cat

[u'everyday cooking',
 u'main dishes',
 u'desserts',
 u'appetizers and snacks',
 u'pasta by shape',
 u'chicken breasts',
 u'asian recipes',
 u'side dishes',
 u'pasta main dishes',
 u'soups, stews and chili',
 u'fruit desserts',
 u'italian recipes',
 u'salad recipes',
 u'bread recipes',
 u'breakfast and brunch',
 u'mexican recipes',
 u'cake recipes',
 u'faceless recipes',
 u'drinks',
 u'cookies',
 u'soup',
 u'trusted brands',
 u'baked and roasted chicken',
 u'dips and spreads',
 u'vegan recipes',
 u'fish recipes',
 u'casseroles',
 u'canned beans and peas',
 u'vanilla extract',
 u'cheese appetizers']

## Map each recipe to a category
since each recipe contains multiple categories, we loop through the categories list for each recipe, and map the recipe to ONE category in the TOP_K_CAT list. If a recipe doesn't belong in any category, we map it to 'Other'

In [338]:
def mapToSingleCategory(arr):
    for item in arr:
        if item.lower() in k_cat:
            return item.lower()
    return 'other'
df_single_cat = df.copy()
df_single_cat['cat_label'] = df_single_cat['categories'].apply(mapToSingleCategory)
df_single_cat['cat_label'].value_counts()

everyday cooking             12170
side dishes                   5847
fruit desserts                4884
salad recipes                 4733
asian recipes                 4434
chicken breasts               4350
breakfast and brunch          3999
italian recipes               3761
appetizers and snacks         3679
desserts                      3633
trusted brands                3416
soup                          3366
bread recipes                 3270
cookies                       3180
dips and spreads              3033
pasta by shape                3030
drinks                        2903
cake recipes                  2490
main dishes                   2264
mexican recipes               2220
soups, stews and chili        2099
fish recipes                  1812
cheese appetizers             1318
casseroles                    1201
baked and roasted chicken      675
pasta main dishes              497
vegan recipes                  440
other                          348
vanilla extract     

## Create Embedding using the unique_ingredients
We have already created a list of unique_ingredients, so we just need to read it in and map each recipe to a one-hot vector. This takes a long time so we processed and stored the word embedding matrix in sparse_recipe_ingredient_matrix.npz. We just need to map those to the categories for training.

In [336]:
# !aws s3 cp s3://RecipeVectors/sparse_recipe_ingredient_matrix.npz .
# https://stackoverflow.com/a/8980156/2491761
def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

recipe_ingredient_matrix = load_sparse_csr("sparse_recipe_ingredient_matrix.npz")
print 'recipe_ingredient_matrix shape:', recipe_ingredient_matrix.shape

svd = TruncatedSVD(n_components=50)
reduced_recipe_ingredient_matrix = svd.fit_transform(recipe_ingredient_matrix)

print type(reduced_recipe_ingredient_matrix)
print 'reduced_recipe_ingredient_matrix shape:', reduced_recipe_ingredient_matrix.shape

recipe_ingredient_matrix shape: (89061, 19013)
<type 'numpy.ndarray'>
reduced_recipe_ingredient_matrix shape: (89061, 50)


In [339]:
df_single_cat['features'] = list(reduced_recipe_ingredient_matrix)
df_single_cat[['features','cat_label']].sample(5)

,features,cat_label
19139,"[1.22050713893, 1.76628974185, 0.0354851312098...",bread recipes
24589,"[1.89889819297, -0.686912104969, -0.4275120361...",breakfast and brunch
1085,"[1.48215031547, 1.68507335044, -0.462875380731...",fruit desserts
25220,"[1.12445043427, 0.796921282012, -0.22731742225...",bread recipes
2125,"[2.50889096518, -1.6057192107, -1.01385053416,...",main dishes


## encode the labels
Here we use sklearn's preprocessing to encode the 30 labels into indices; the final softmax layer in our classification model will be a vector of length 30, corresponding to the probability of each label for a given input. 

In [343]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df_single_cat['cat_label'])
print le.classes_

[u'appetizers and snacks' u'asian recipes' u'baked and roasted chicken'
 u'bread recipes' u'breakfast and brunch' u'cake recipes' u'casseroles'
 u'cheese appetizers' u'chicken breasts' u'cookies' u'desserts'
 u'dips and spreads' u'drinks' u'everyday cooking' u'fish recipes'
 u'fruit desserts' u'italian recipes' u'main dishes' u'mexican recipes'
 'other' u'pasta by shape' u'pasta main dishes' u'salad recipes'
 u'side dishes' u'soup' u'soups, stews and chili' u'trusted brands'
 u'vanilla extract' u'vegan recipes']


In [342]:
df_single_cat['label'] = le.transform(df_single_cat['cat_label'])
df_final = df_single_cat[['features','label']]
df_final.sample(5)

,features,label
7506,"[0.251008849796, 0.454623460639, 0.12009385247...",15
4543,"[1.03191109934, 2.11129561861, -0.024014297480...",10
10785,"[2.22011559852, -0.470587151997, 0.23649744678...",6
8867,"[4.73585551979, -0.43068586289, -0.31929311014...",13
10511,"[0.398673862123, -0.135033842398, 0.0708437121...",22


In [346]:
# test the encoding works
le.inverse_transform([6,12,0])

array([u'casseroles', u'drinks', u'appetizers and snacks'], dtype=object)

## Finalized Dataset
The final dataset is split into Train/Test sets,  we can now pipe this through a CNN model with softmax layer. 

In [245]:
from sklearn.model_selection import train_test_split
features = np.array([np.array(row) for row in df_final['features'].values])
labels = df_final['label'].values

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20)
print 'X_train shape:', X_train.shape
print 'y_train shape:', y_train.shape
print 'X_test shape:', X_test.shape
print 'y_test shape:', y_test.shape
# print 'sample X:', np.random.choice(X_train, 3)

 X_train shape: (71248, 50)
y_train shape: (71248,)
X_test shape: (17813, 50)
y_test shape: (17813,)


In [323]:
# global max pooling : https://www.quora.com/What-is-global-average-pooling
# CNN for text classification: https://arxiv.org/pdf/1408.5882.pdf

from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.utils import to_categorical


def trainKerasModel_conv(X,Y):
    print('Shape of data tensor:', X.shape)
    print('Shape of label tensor:', Y.shape)

    print('Training model.')
    BATCH_SIZE = 50
    # train a 1D convnet with global maxpooling
#     inp = Input(shape=(None,MAX_SEQUENCE_LENGTH,))
    model = Sequential()
    model.add(Dense(128, input_shape=(MAX_SEQUENCE_LENGTH), activation='relu'))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(35))  # global max pooling
    model.add(Conv1D(128, 1, activation='relu'))
#     model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(len(labels), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])
    print model.summary()
    model.fit(X, Y, epochs=2, batch_size=BATCH_SIZE)
    # Final evaluation of the model
    return model

conv_model = trainKerasModel_conv(X_train, y_train)

('Shape of data tensor:', (71248, 50))
('Shape of label tensor:', (71248,))
Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 50, 128)           256       
_________________________________________________________________
conv1d_95 (Conv1D)           (None, 46, 128)           82048     
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 1, 128)            0         
_________________________________________________________________
conv1d_96 (Conv1D)           (None, 1, 128)            16512     
_________________________________________________________________
dense_76 (Dense)             (None, 1, 128)            16512     
_________________________________________________________________
dense_77 (Dense)             (None, 1, 89061)          11488869  
Total params: 11,604,197
Trainable params: 11,604,

ValueError: Error when checking input: expected dense_75_input to have 3 dimensions, but got array with shape (71248, 50)

# Generative Model 3
Let's do something fun and generate the recipes from scratch :D

In [347]:
df.columns.values

array([u'categories', u'cookingTime', u'description', u'ingredients',
       u'instructionSteps', u'name', u'rating', u'ratingCount', u'url',
       'cookingTimeMinutes', 'cleanedIngredients', 'allCategories',
       'category_cluster_index'], dtype=object)

In [362]:

import re
def processCategories(arr):
    arr = [re.sub(r'[\xa0\x99]', ' ', k.encode('ascii', 'ignore')) for k in arr]
    return '<category>' + ','.join(arr) + '</category>'
def processIngredients(arr):
    arr = [re.sub(r'[\xa0\x99]', ' ', k.encode('ascii', 'ignore')) for k in arr]
    return '<ingredients>' + ','.join(arr) + '</ingredients>'
def processName(text):
    return '<name> '+ text+' </name>'
def processDesc(text):
    if text == text:
        text = re.sub(r'[\xa0\x99]', ' ', text.encode('ascii', 'ignore'))
        return '<desc> '+ str(text) +' </desc>'
    return '<no_desc>'
def processInstructions(arr):
    if not arr == arr:
        return '<no_inst>'
    retStr = '<inst> '
    for i, inst in enumerate(arr):
        inst = re.sub(r'[\xa0\x99]', ' ', inst.encode('ascii', 'ignore'))
        retStr += (str(i) +' : '+inst + ' ')
    retStr += ' </inst>'
    return retStr
def getRating(text):
    return '<rating> ' + str(text) + ' </rating>' 
def addRowsTogether(row):
    return ' '.join(row[['ingredients', 'name', 'description','instructionSteps','categories','rating']])


df_recipe_generator = df.copy()
df_recipe_generator['categories'] = df_recipe_generator['categories'].apply(processCategories)
df_recipe_generator['name'] = df_recipe_generator['name'].apply(processName)
df_recipe_generator['ingredients'] = df_recipe_generator['ingredients'].apply(processIngredients)
df_recipe_generator['description'] = df_recipe_generator['description'].apply(processDesc)
df_recipe_generator['rating'] = df_recipe_generator['rating'].apply(getRating)
df_recipe_generator['instructionSteps'] = df_recipe_generator['instructionSteps'].apply(processInstructions)

df_recipe_generator['total_recipe'] = df_recipe_generator.apply(addRowsTogether, axis=1)


In [363]:
df_recipe_generator['total_recipe'].sample(5)

1935     <ingredients>2 cups fresh broccoli florets,1 t...
20691    <ingredients>3 cups 1-inch pieces rhubarb,1 1/...
2132     <ingredients>1 tablespoon vegetable oil,4 bone...
11233    <ingredients>cooking spray,1 cup almond flour,...
27386    <ingredients>1 cup packed brown sugar,1/4 cup ...
Name: total_recipe, dtype: object

In [371]:
data = df_recipe_generator['total_recipe'].values
chars = list(set(data))
VOCAB_SIZE = len(chars)
SEQ_LENGTH = 200

In [372]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

In [ ]:
X = np.zeros((len(data)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(data)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(data)/SEQ_LENGTH):
    X_sequence = data[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [ ]:
def concatIngredients(arr):
    return ','.join(arr).encode('ascii', 'ignore')

def processKerasModel3(df_):
    embedding_length = 100
    top_words = 10000
    df = df_.copy()
    df['features'] = df['ingredients'].apply(concatIngredients)
    df['label'] = df['category_cluster_index']
    df = df[df['label'].astype(str) != 'nan']
    le = preprocessing.LabelEncoder()
    features = df['features'].values
    labels = df['label'].values
    
    le.fit(np.unique(labels))
    print list(le.classes_), 'num of labels:',len(np.unique(labels))
    labels = le.transform(labels) 
    print labels[:10]
#     X_train, Y_train, X_test, Y_test = ut.simpleSplit(features, labels)

    tokenizer = Tokenizer(nb_words=top_words)
    tokenizer.fit_on_texts(features)
    sequences = tokenizer.texts_to_sequences(features)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=embedding_length)

#     labels = np_utils.to_categorical(np.asarray(labels))
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    # split the data into a training set and a validation set
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices]
    labels = labels[indices]
    nb_validation_samples = int(0.2 * data.shape[0])

    X_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    X_test = data[-nb_validation_samples:]
    y_test = labels[-nb_validation_samples:]
    
    embedding_vecor_length = 100
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=embedding_length))
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
#     model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(len(np.unique(labels)), activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])
    print(model.summary())
    
    model.fit(X_train, y_train, epochs=10, batch_size=100)
    # Final evaluation of the model
    return model, X_test, y_test

rnn_generative_model, X_test, y_test = processKerasModel3(df_recipe_generator)

In [ ]:

def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)